A Dataracing 2022 versenyén hazai vállalatok adatainak elemzésére nyílik lehetőséged különféle data science eszközökkel. A feladat a hazai vállalatok egy csoportjának exportteljesítményének előrejelzéséről szól, az ehhez az anonimizált adatokat a verseny egyik társszervezője, a Magyar Nemzeti Bank biztosította. Az adatelemzési cél, hogy az egyes cégek múltbeli beszámoló adatai, illetve egyéb statisztikái mérőszámai alapján a versenyzők gépi tanulási eszközökkel előre jelezzék az egyes cégek következő évi exportból származó bevételét.

Mennyire jelezhető előre az egyes cégek üzleti teljesítménye? Milyen szinten lehet becsülni a múltbeli adatok alapján a várható export értékét? Milyen elemzési trükkök vethetők be egy ilyen adathalmaz esetén? Mely algoritmusok a legalkalmasabbak az adott esetben? Ilyen és ehhez hasonló kérdésekre keresi a választ az MNB, mely megválaszolásához a versenyen részvevőként Te is hozzájárulhatsz. A legjobb három megoldás készítő számára magas pénzjutalmak kerülnek kifizetésre.

Az adathalmazról

A verseny során hazai vállalatok adatait elemezzük. Az adathalmazban több mint 30 ezer cég különféle adatait találjuk, a cégek székhelyétől kezdve egészen a létszám és árbevétel információkig.

Az adathalmazban egy-egy sor egy-egy céghez tartozik. A rendelkezésre álló értékek a 2014-2016 időszakban minden évben felvételre kerültek, ezt az időszakot teljes mértékben rendelkezésre bocsájtjuk. Az adathalmazban előfordulhat hiányzó érték, ennek kezelését a versenyzőre bízzuk.

A cél felügyelt gépi tanulási módszerek segítségével előre jelezni az “target_reg” névvel ellátott oszlop (célváltozó) értékét. Ez a változó a 2017-es export értékét reprezentálja ezer Forintban.

A versenyen a célváltozó előre jelzéséhez elsősorban a publikált adathalmaz további oszlopait (bemenő változók) lehet használni.

Minden adat típust használjunk fel!

adatok: ’https://raw.githubusercontent.com/karsarobert/Machine_Learning_2023/main/train.csv’

metrika: Mean Absolute Error MAE

célváltozó: target_reg

Adatok felosztása az alábbiak szerint:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

MAE-t a X_test-en mérjük!

Készítette: Szakonyi Bence (C7BJJP), Tomor Máté (GNF0N4)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Adatok beolvasása
data_url = 'https://raw.githubusercontent.com/karsarobert/Machine_Learning_2023/main/train.csv'
df = pd.read_csv(data_url)

# Adatok felosztása tanító- és teszthalmazra
X = df.drop(columns=['target_reg'])
y = df['target_reg']



# Hiányzó értékek kezelése és kategorikus oszlopok átalakítása
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Numerikus oszlopok kezelése
imputer = SimpleImputer(strategy='mean')
X[numeric_cols] = imputer.fit_transform(X[numeric_cols])

# Kategorikus oszlopok kezelése
X = pd.get_dummies(X, columns=categorical_cols)

# Adatok felosztása tanító- és teszthalmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Modell létrehozása
model = RandomForestRegressor()

# Paraméterkeresési tartomány meghatározása
param_grid = {'n_estimators': [100, 200, 500],
              'max_depth': [None, 5, 10]}

# Paraméterkeresés az almintán
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3)
grid_search.fit(X_train[:1000], y_train[:1000])

# Legjobb paraméterek kinyerése
best_params = grid_search.best_params_

# Modell létrehozása a teljes X_train adatkészlettel és a megtalált paraméterekkel
model = RandomForestRegressor(**best_params)
model.fit(X_train, y_train)

# Előrejelzés a teszthalmazon
y_pred = model.predict(X_test)

# Mean Absolute Error (MAE) számítása
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


Mean Absolute Error: 54254.323985403134


#JAVITÁS

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Adatok beolvasása
data_url = 'https://raw.githubusercontent.com/karsarobert/Machine_Learning_2023/main/train.csv'
df = pd.read_csv(data_url)

In [ ]:
# Felosztás a független változók (X) és a célváltozó (y) között
X = df.drop(columns=['target_reg'])
y = df['target_reg']

In [ ]:
# Felosztás tréning- és teszthalmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Ellenőrzés
print("Tréning adatok mérete:", X_train.shape)
print("Teszt adatok mérete:", X_test.shape)

Tréning adatok mérete: (19908, 131)
Teszt adatok mérete: (13272, 131)


In [ ]:
# Numerikus oszlopok kiválasztása
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns

# Kategórikus oszlopok kiválasztása
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Tréning adatok szétválasztása numerikus és kategórikus részekre
X_train_numeric = X_train[numeric_cols]
X_train_categorical = X_train[categorical_cols]

# Teszt adatok szétválasztása numerikus és kategórikus részekre
X_test_numeric = X_test[numeric_cols]
X_test_categorical = X_test[categorical_cols]

# Ellenőrzés
print("Numerikus oszlopok (tréning):", numeric_cols)
print("Kategórikus oszlopok (tréning):", categorical_cols)
print("Numerikus adatok (tréning) mérete:", X_train_numeric.shape)
print("Kategórikus adatok (tréning) mérete:", X_train_categorical.shape)
print("Numerikus adatok (teszt) mérete:", X_test_numeric.shape)
print("Kategórikus adatok (teszt) mérete:", X_test_categorical.shape)

Numerikus oszlopok (tréning): Index(['id', 'KATEGORIKUS_VALTOZO_2014', 'KATEGORIKUS_VALTOZO_2015',
       'KATEGORIKUS_VALTOZO_2016', 'aktelh_2014', 'aktelh_2015', 'aktelh_2016',
       'aktsajt_2014', 'aktsajt_2015', 'aktsajt_2016', 'alakul_ev',
       'arbevert_2014', 'arbevert_2015', 'arbevert_2016', 'arbevexp_2014',
       'arbevexp_2015', 'arbevexp_2016', 'befpue_2014', 'befpue_2015',
       'befpue_2016', 'begyeb_2014', 'begyeb_2015', 'begyeb_2016',
       'celtart_2014', 'celtart_2015', 'celtart_2016', 'eredadel_2014',
       'eredadel_2015', 'eredadel_2016', 'eredadoz_2014', 'eredadoz_2015',
       'eredadoz_2016', 'eredpenz_2014', 'eredpenz_2015', 'eredpenz_2016',
       'eredtar_2014', 'eredtar_2015', 'eredtar_2016', 'ereduzem_2014',
       'ereduzem_2015', 'ereduzem_2016', 'ertpapir_2014', 'ertpapir_2015',
       'ertpapir_2016', 'hoskot01_2014', 'hoskot01_2015', 'hoskot01_2016',
       'hoskot_2014', 'hoskot_2015', 'hoskot_2016', 'immat_2014', 'immat_2015',
       'immat_20

In [ ]:
# Numerikus adatok imputálása
numeric_imputer = SimpleImputer(strategy='mean')
X_train_numeric_imputed = numeric_imputer.fit_transform(X_train_numeric)
X_test_numeric_imputed = numeric_imputer.transform(X_test_numeric)

# Kategórikus adatok imputálása
categorical_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical_imputed = categorical_imputer.fit_transform(X_train_categorical)
X_test_categorical_imputed = categorical_imputer.transform(X_test_categorical)

In [ ]:
# Kategórikus adatok kódolása
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_encoded = encoder.fit_transform(X_train_categorical_imputed)
X_test_categorical_encoded = encoder.transform(X_test_categorical_imputed)

In [ ]:
# Numerikus és kategórikus adatok összefűzése
X_train_combined = pd.concat([pd.DataFrame(X_train_numeric_imputed, columns=numeric_cols),
                              pd.DataFrame(X_train_categorical_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_cols))],
                             axis=1)

X_test_combined = pd.concat([pd.DataFrame(X_test_numeric_imputed, columns=numeric_cols),
                             pd.DataFrame(X_test_categorical_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_cols))],
                            axis=1)

# Ellenőrzés
print("Összefűzött tréning adatok mérete:", X_train_combined.shape)
print("Összefűzött teszt adatok mérete:", X_test_combined.shape)

Összefűzött tréning adatok mérete: (19908, 6134)
Összefűzött teszt adatok mérete: (13272, 6134)


In [ ]:
# Alminták létrehozása a paraméterkereséshez
X_train_sample = X_train_combined.sample(frac=0.2, random_state=42)
y_train_sample = y_train.sample(frac=0.2, random_state=42)

In [ ]:
# Modell létrehozása
model = RandomForestRegressor()

In [ ]:
# Paraméterkeresési tartomány meghatározása
param_grid = {'n_estimators': [100, 200, 500],
              'max_depth': [None, 5, 10]}

In [ ]:
# Paraméterkeresés az almintán
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=3)

# Modell illesztése az almintán és paraméterkeresés elvégzése
grid_search.fit(X_train_sample, y_train_sample)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 5, 10],
                         'n_estimators': [100, 200, 500]},
             scoring='neg_mean_absolute_error')

In [ ]:
# Legjobb paraméterek kinyerése
best_params = grid_search.best_params_

print(best_params)

{'max_depth': 10, 'n_estimators': 500}


In [ ]:
# Modell létrehozása a teljes X_train adatkészlettel és a megtalált paraméterekkel
model = RandomForestRegressor(**best_params)
model.fit(X_train_combined, y_train)

RandomForestRegressor(max_depth=10, n_estimators=500)

In [ ]:
# Előrejelzés a teszthalmazon
y_train_pred = best_model.predict(X_train_combined)
y_test_pred = best_model.predict(X_test_combined)

In [ ]:
# Teljes adathalmazon való illesztés és értékelés
y_train_pred = best_model.predict(X_train_combined)
y_test_pred = best_model.predict(X_test_combined)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Eredmények kiíratása
print("Tréning adatok MAE értéke:", mae_train)
print("Teszt adatok MAE értéke:", mae_test)

Tréning adatok MAE értéke: 90536.22925406872
Teszt adatok MAE értéke: 98789.58584915611


In [ ]:
# Legjobb modell kiválasztása
best_model = grid_search.best_estimator_

In [ ]:
# Teljes adathalmazon való illesztés és értékelés
y_train_pred = best_model.predict(X_train_combined)
y_test_pred = best_model.predict(X_test_combined)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# Eredmények kiíratása
print("Tréning adatok MAE értéke:", mae_train)
print("Teszt adatok MAE értéke:", mae_test)

Tréning adatok MAE értéke: 90536.22925406872
Teszt adatok MAE értéke: 98789.58584915611
